In [6]:
import torch

if torch.cuda.is_available():
  print('Using GPU')
  device = 'cuda'

Using GPU


In [7]:
import os
from os import path
from argparse import ArgumentParser
import shutil

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image

from inference.data.test_datasets import LongTestDataset, DAVISTestDataset, YouTubeVOSTestDataset
from inference.data.mask_mapper import MaskMapper
from model.network import XMem
from inference.inference_core import InferenceCore

from progressbar import progressbar

In [8]:
torch.set_grad_enabled(False)

# default configuration
config = {
    'top_k': 30,
    'mem_every': 5,
    'deep_update_every': -1,
    'enable_long_term': True,
    'enable_long_term_count_usage': True,
    'num_prototypes': 128,
    'min_mid_term_frames': 5,
    'max_mid_term_frames': 10,
    'max_long_term_elements': 10000,
}

network = XMem(config, '/home/fyp/XMem/saves/XMem.pth').eval().to(device)

Hyperparameters read from the model weights: C^k=64, C^v=512, C^h=64
Single object mode: False


In [14]:
import numpy as np
import cv2
mask_name = "/home/fyp/mask_n.jpg"
mask = np.array(Image.open(mask_name))
for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
        if mask[i][j] > 200:
            mask[i][j] = 255
        else:
            mask[i][j] = 0
print(np.unique(mask))
num_objects = len(np.unique(mask)) - 1
num_objects
print(mask)
mask.shape

[  0   1   2   3   4   5   6   7 248 249 250 251 252 253 254 255]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


(720, 1280)

In [13]:

import cv2
from inference.interact.interactive_utils import image_to_torch, index_numpy_to_one_hot_torch, torch_prob_to_numpy_mask, overlay_davis

torch.cuda.empty_cache()

processor = InferenceCore(network, config=config)
processor.set_all_labels(range(1, num_objects+1)) # consecutive labels
frame = cv2.imread("/home/fyp/image.jpg")


with torch.cuda.amp.autocast(enabled=True):

    # convert numpy array to pytorch tensor format
    frame_torch, _ = image_to_torch(frame, device=device)
    # initialize with the mask
    mask_torch = index_numpy_to_one_hot_torch(mask, num_objects+1).to(device)
    # the background mask is not fed into the model
    prediction = processor.step(frame_torch, mask_torch[1:])
    # argmax, convert to numpy
    prediction = torch_prob_to_numpy_mask(prediction)

frame.shape


(720, 1280, 3)

In [10]:
import cv2
from inference.interact.interactive_utils import image_to_torch, index_numpy_to_one_hot_torch, torch_prob_to_numpy_mask, overlay_davis

torch.cuda.empty_cache()

processor = InferenceCore(network, config=config)
processor.set_all_labels(range(1, num_objects+1)) # consecutive labels
cap = cv2.VideoCapture(video_name)

# You can change these two numbers
frames_to_propagate = 200
visualize_every = 20

current_frame_index = 0

with torch.cuda.amp.autocast(enabled=True):
  while (cap.isOpened()):
    # load frame-by-frame
    _, frame = cap.read()
    if frame is None or current_frame_index > frames_to_propagate:
      break

    # convert numpy array to pytorch tensor format
    frame_torch, _ = image_to_torch(frame, device=device)
    if current_frame_index == 0:
      # initialize with the mask
      mask_torch = index_numpy_to_one_hot_torch(mask, num_objects+1).to(device)
      # the background mask is not fed into the model
      prediction = processor.step(frame_torch, mask_torch[1:])
    else:
      # propagate only
      prediction = processor.step(frame_torch)

    # argmax, convert to numpy
    prediction = torch_prob_to_numpy_mask(prediction)

    if current_frame_index % visualize_every == 0:
      visualization = overlay_davis(frame, prediction)
      display(Image.fromarray(visualization))

    current_frame_index += 1

NameError: name 'video_name' is not defined